In [1]:
# Импорт библиотек
import pandas as pd
from datetime import datetime

In [2]:
# Чтение файла 'productstar_df.csv'
df_path = 'productstar_df.csv'
df = pd.read_csv(df_path)
df

,id,shipped_at,order_id,item_total,order_state,store_id,retailer_id,promo_total,total_weight,completed_at,city_id,user_id,item_count
0,0,2018-05-20 01:42:14+00:00,557,3243.119268,complete,75,8,-200.0,6085,2018-05-19 09:16:54+00:00,1,9016,13
1,1,NaN,1097,4590.108329,canceled,62,8,0.0,3000,2017-01-25 05:06:43+00:00,1,9052,2
2,2,2017-06-14 03:26:44+00:00,3103,12018.548693,complete,62,8,0.0,179750,2017-06-12 03:52:36+00:00,1,9041,46
3,3,2017-06-12 00:16:40+00:00,3112,2467.022803,complete,62,8,0.0,9100,2017-06-11 01:01:29+00:00,1,9041,6
4,4,2017-05-31 05:23:53+00:00,3201,1461.915164,complete,68,15,0.0,5948,2017-05-30 22:59:38+00:00,1,9041,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474954,474954,2019-01-05 23:01:39+00:00,3642207,2261.725542,complete,63,8,0.0,39100,2019-01-05 11:07:53+00:00,1,67812,6
474955,474955,NaN,3642208,3234.739393,canceled,75,8,0.0,12300,2018-12-16 22:49:59+00:00,1,144725,11
474956,474956,2019-01-23 05:19:03+00:00,3642213,3621.669987,complete,75,8,0.0,48972,2019-01-22 11:44:37+00:00,1,22096,9
474957,474957,2018-12-26 09:02:33+00:00,3642227,3089.668218,complete,73,8,0.0,25710,2018-12-26 04:38:16+00:00,1,9424,9


In [3]:
# Преобразование данных в datetime
df['shipped_at'] = pd.to_datetime(df['shipped_at'])
df['completed_at'] = pd.to_datetime(df['completed_at'])

In [4]:
df['year'] = df['completed_at'].dt.year
df['month'] = df['completed_at'].dt.month

In [5]:
# Создание датафрейма с выполненными заказами
df_compl = df[df['order_state'] == 'complete']
# df_compl

1 Найти разницу в продажах между декабрём 2018 и декабрём 2019 в процентах.

In [18]:
# Создание датафрейма с продажами по месяцам
df1 = df_compl[['year', 'month', 'item_total']].groupby(['year', 'month'], as_index = True).sum()
# df1

In [21]:
# Вычисление разницы в продажах
df_2018 = df1['item_total'][(2018, 12)]
df_2019 = df1['item_total'][(2019, 12)]
# df_2018
delta = round((df_2019 - df_2018)/df_2018 * 100)
print(delta)

329


2 Найти разницу в продажах между средним и медианой за 2018.

In [8]:
# Создание датафрейма за 2018
df2 = df_compl[df_compl['completed_at'].dt.year == 2018]
# df2

In [19]:
df_2018_by_id = df2.groupby('user_id').sum(numeric_only = True)

In [20]:
# Поиск среднего и медианы
sr = df_2018_by_id['item_count'].mean()
med = df_2018_by_id['item_count'].median()

# Разница между медианой и средним по кол-ву продаж в %
round((sr - med)/med * 100)

141

3 Найти наименее прибыльный месяц по продажам за 2018 год (с учётом издержек).

In [12]:
# Создание датафрейма со всеми заказами за 2018
df3 = df[df['completed_at'].dt.year == 2018]

# Создание датафрейма со всеми совершёнными заказами за 2018
df4 = df_compl[df_compl['completed_at'].dt.year == 2018]

In [13]:
# Расчёт издержек
df3.loc[df3['order_state'] == 'complete', 'издержки'] = 0.15 * df3['total_weight']
df3.loc[df3['order_state'] == 'canceled', 'издержки'] = 0.30 * df3['total_weight']
# df3

C:\Users\Sorochuk\AppData\Local\Temp\ipykernel_11184\2399250170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[df3['order_state'] == 'complete', 'издержки'] = 0.15 * df3['total_weight']


In [14]:
df_iz = df3[['month', 'издержки']].groupby(['month'], as_index = True).sum()
# df_iz

In [15]:
df_item = df4[['month', 'item_total']].groupby(['month'], as_index = True).sum()
# df_item

In [16]:
df_itog = pd.concat([df_item, df_iz], axis=1)
# df_itog
df_itog['итоговая прибыль'] = df_itog['item_total'] - df_itog['издержки']
df_itog.sort_values(by=['итоговая прибыль'])

,item_total,издержки,итоговая прибыль
month,,,
6,3.104431e+07,39310605.00,-8.266295e+06
1,3.216289e+07,38776632.30,-6.613747e+06
8,3.611081e+07,41878537.80,-5.767724e+06
5,3.526819e+07,40276701.30,-5.008516e+06
7,3.024040e+07,34297243.95,-4.056844e+06
2,3.147069e+07,35291974.35,-3.821287e+06
3,3.744271e+07,41030466.90,-3.587757e+06
4,3.459899e+07,37855123.95,-3.256132e+06
9,4.491716e+07,43155445.65,1.761718e+06
